# Function Definitions
## Age, Gender, State, Comorbidies

In [160]:
# Libraries
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import random
from statistics import mode
import numpy as np
import pylab

In [161]:
# Global variables
COUNT = 1000

# Dictionary to map states to their abbreviations
US_STATES = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'New York City': 'NYC',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# US population
US_POP = 328.2e6

# NYC population
NYC_POP = 8_740_000

# Occupations
OCCUPATIONS = {
    'Health Care Worker': 100,
    'First Responder': 90,
    'Corrections Workers': 90,
    'Food and Agriculture': 90,
    'Grocery Store Workers': 90,
    'Education': 90,
    'U.S. Postal Service Workers': 90,
    'Public Transit Workers': 90,
    'Manufacturing': 90,
    'Transportation and Logistics': 80,
    'Food Service': 80,
    'Energy': 80,
    'Water and Wastewater': 80,
    'Shelter and Housing': 80,
    'IT and Communication': 80,
    'News Media': 80,
    'Public Safety': 80,
    'Public Health Workers': 80,
    'Finance': 80,
    'Legal': 80,
    'Other': 0}

# Weights for final score
WEIGHTS = [0.35, 0.25, 0.20, 0.15, 0.05]

### Age Functions

In [162]:
# Takes input age, prints custom bracket, mult, and df category  
def age_score(df):
    """Determines a user's vaccine prioriry score based on the age they input
    Input: A dataframe of 
    Output: The score for the user's age bracket as a float (from 0 to 100)
    Note: needs the 'df_age_cont' df with Scores and Thresholds to run"""
    
    # Takes user age input and makes sure it's a number
    num_in = False
    while num_in == False:
        age = input("What is your age in years? ")
        try:
            age = float(age)
            num_in = True
        except ValueError:
            print(f"Please input a number (integer or float) (entered type was {type(age)})")
            num_in = False
    
    # Determines the user's score by itterating through threshold ages in df
    decimals = 5
    score = df['Score'].iloc[-1] # Rewritten in loop if user isn't in the highest age group
    for r in range(len(df['Threshold'])):
        if age < df['Threshold'].iloc[r]:
            score = df['Score'].iloc[r-1]
            break
    
    return round(score, decimals)

### Gender/Occupation Functions

In [163]:
def occupation_score():
    ''' Provides occupation contribution to final vaccine score'''
    
    print(list(OCCUPATIONS.keys()))
    
    occupation_question = input('Indicate your occupation based on the list ')
    if occupation_question != 'Other':
        score = OCCUPATIONS[occupation_question]
    else:
        score = other_occupation_score()
    
    return score

In [164]:
def other_occupation_score():
    ''' Outputs the occupation score contribution for individuals who answered "Other" '''

    # Remote question
    remote_question = input('Do you work in-person? [y/n] \n')
    while True:
    
        if remote_question == 'n':
            contact = 0
            break
            
        # If not remote
        elif remote_question == 'y':
        
            contact_question = input('How many people in contact with per week? (#)\n')
            num_contact = [[0],[100],[contact_question]]
            scaler = MinMaxScaler(feature_range = (0,100))
            x = scaler.fit(num_contact)
            contact = x.transform(num_contact)[-1]
            break
        else:
            remote_question = input(' Please enter a valid input [y/n]')

    # Exposure question
    exposure_question = input('Are you regularly exposed to individuals who are ' \
                               'at risk for COVID-19? [y/n] \n')
 
    while True:
        if exposure_question == 'y':
            exposure = 100
            break
        elif exposure_question == 'n':
            exposure = 0
            break
        else:
            exposure_question = input(' Please enter a valid input [y/n]')
    
    # Calculate occupation score
    occupation_avg = 0.35 * contact + 0.65 * exposure
    
    return float(occupation_avg)

In [165]:
def gender_score():
    ''' Calculates the gender contribution to the overall vaccine score '''
    
    # Gender Questionnaire
    gender_question = input('Please identify your biological sex [m/f]')
    
    if gender_question == 'm' or gender_question == 'M':
        gender_score = gender_df['% Deaths']['Male']
    
    elif gender_question == 'f' or gender_question == 'F':
        gender_score = gender_df['% Deaths']['Female']
        
    return gender_score

### State Functions

In [166]:
def fix_consent(column, row):
    '''
    param: column(string), row(series)
    return: float
    Does: Corrents records that improperly store the total deaths and cases
    '''

    # Generates the proper column names to use for conditional
    consent = f'consent_{column}'
    total = f'tot_{column}'
    prob = f'prob_{column}'

    # Checks to see if row is stored improperly and then corrects it
    if row[consent] == 'Agree':
        return row[total] - row[prob]

    else:
        return row[total]

In [167]:
def count_stats(subset, stat):
    '''
    param: subset(dataframe), stat(string)
    return: stat_count(float)
    Does: Gets the proper stat measurment from the df subset 
    '''

    # Finds last entry in subset and then calculates relevant stat
    stat_count = subset[stat].iloc[-1]
    stat_count = round(stat_count/len(subset), 2)

    return stat_count

In [168]:
def scale_values(values, scaler=MinMaxScaler):
    '''
    param: values(list), scaler(function)
    return: list
    Does: Scales the values according to the scaling function passed
    '''

    # Prepares the scaler passed along with data
    values = list(map(lambda x: [x], values))
    scaler = scaler(feature_range=(0,100))

    # Scales the data and then returns it
    scaler.fit(values)
    values = scaler.transform(values)
    return [value[0] for value in values]

In [169]:
def pop_percent(total, state_pop):
    '''
    param: total(int), state_pop(int)
    return: percent
    Does: Finds the percentage of the country that a state accounts for and
    then returns it
    '''

    # Finds the percent of population for state that was passed
    state_population = state_pop['Population']
    percent = state_population/total

    return percent

In [170]:
def generate_list(percents, count):
    '''
    param: percents(dict), count(int)
    return: random_lst(list)
    Does: Creates a shuffled list
    distribution
    '''

    # Iterates through the list of percents generated before
    random_lst = []
    for key, value in percents.items():
        iterate = int(round(count * value))

    # Adds states to list relative to its total percent
        for num in range(iterate):
            random_lst.append(key)

    # Adjusts list size for rounding errors
    most_common = mode(random_lst)
    while len(random_lst) < count:
        random_lst.append(most_common)

    # Shuffles the list and then returns it
    random.shuffle(random_lst)
    
    while len(random_lst) > count:
        random_lst = random_lst[:-1]
        
    return random_lst

In [171]:
def menu(score_dict):
    '''
    param: final_scores(list)
    return: personal_info(tuple)
    Does: Prompts user for info and then returns it in a tuple
    '''
    # Prints out the possible inputs
    states = [state for state in score_dict.keys()]
    states.sort()
    for value in states:
        print(value)

    # Prompts user for their home state
    state_input = input('Enter the abbreviation that corresponds '
                          'to your home state using the list above.\n')

    # Checks to make sure the input is valid and then loops until True
    while state_input not in states:
        state_input = input('Please enter a valid input'
                              ' from the list above.\n')

    # Returns the response to the question above 
    personal_info = score_dict[state_input]
    return personal_info

In [172]:
def draw_bar(title, xlabel, ylabel, data):
    '''
    param: title(string), xlabel(string), y_label(string), data(dict)
    return: N/A
    Does: Uses the data passed to draw a bar chart
    '''

    # Creates data for the bar chart
    x = list(data.keys())[42:]
    y = list(data.values())[42:]

    # Creates bar chart        
    plt.bar(x=x, height=y, color='coral')

    # Adds proper labels to the bar chart and then shows it
    plt.xticks(label=xlabel, rotation=270)
    plt.ylabel(ylabel=ylabel)
    plt.title(title)
    plt.show()

### Comorbidities Function

In [173]:
def comorbidities(lst, dictionary):
    '''Gives score for comorbidity section for vaccine distribution'''
    
    for L in lst:
        print(L)
    choice = input('Do you have a condition on this list? (y or n)\n')

    if choice.lower() == 'y':
        comorbidity = input('Which one?\n')
        score = dictionary[comorbidity]
        return score
    elif choice.lower() == 'n':
        score = 0
        return score

### Final score calculation

In [174]:
def final_score(all_scores, WEIGHTS):
    ''' Calculates the final vaccine score for an individual
        Input: list of scores from each category, category weight contribution list
        Output: final vaccine score '''
    
    final = 0
    for i in range(len(all_scores)):
        final += all_scores[i] * WEIGHTS[i]
        
    return final